In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import holidays
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression,ElasticNet
from sklearn.preprocessing import StandardScaler

In [3]:
df = pd.read_csv('flights')

D:\big data\anaconda3\envs\LH\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [21]:
df['arr_delay'].describe()

count    1.561574e+07
mean     5.475932e+00
std      4.987401e+01
min     -1.290000e+03
25%     -1.500000e+01
50%     -6.000000e+00
75%      8.000000e+00
max      2.973000e+03
Name: arr_delay, dtype: float64

In [6]:
def getmonth(x):
    return x.split(sep='-')[1]
df['month'] = df['fl_date'].apply(getmonth)

In [23]:
df.groupby(['month'])['arr_delay'].mean()

month
01     3.958788
02     6.745096
03     2.818773
04     4.159131
05     6.511144
06    10.414444
07     8.977515
08     8.898890
09     1.708452
10     2.853585
11     2.993607
12     5.110636
Name: arr_delay, dtype: float64

In [20]:
df_01.describe()

,Unnamed: 0,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,dep_delay,crs_arr_time,arr_delay,cancelled,crs_elapsed_time,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
count,1.239868e+06,1.239868e+06,1.239868e+06,1.239868e+06,1.239868e+06,1.220267e+06,1.239868e+06,1.217645e+06,1.239868e+06,1.239860e+06,1.239868e+06,224852.000000,224852.000000,224852.000000,224852.000000,224852.000000
mean,9.183284e+06,2.706668e+03,1.268984e+04,1.268994e+04,1.326527e+03,9.830395e+00,1.497141e+03,3.958788e+00,1.577507e-02,1.398926e+02,7.654198e+02,20.933281,4.550922,16.304369,0.083775,26.351742
std,4.656840e+06,1.885516e+03,1.520026e+03,1.520074e+03,4.826623e+02,4.847698e+01,5.048587e+02,5.091909e+01,1.246043e-01,7.298561e+01,5.819673e+02,66.119720,37.320588,37.154554,3.157289,53.091941
min,4.086620e+06,1.000000e+00,1.013500e+04,1.013500e+04,1.000000e+00,-1.280000e+03,1.000000e+00,-1.290000e+03,0.000000e+00,-9.000000e+01,3.100000e+01,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.396587e+06,1.068000e+03,1.129200e+04,1.129200e+04,9.200000e+02,-6.000000e+00,1.112000e+03,-1.600000e+01,0.000000e+00,8.800000e+01,3.370000e+02,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.339764e+07,2.246000e+03,1.288900e+04,1.288900e+04,1.320000e+03,-3.000000e+00,1.520000e+03,-7.000000e+00,0.000000e+00,1.200000e+02,6.030000e+02,0.000000,0.000000,4.000000,0.000000,0.000000
75%,1.370761e+07,4.317000e+03,1.405700e+04,1.405700e+04,1.730000e+03,6.000000e+00,1.918000e+03,7.000000e+00,0.000000e+00,1.700000e+02,1.008000e+03,18.000000,0.000000,20.000000,0.000000,31.000000
max,1.401758e+07,9.401000e+03,1.621800e+04,1.621800e+04,2.359000e+03,2.007000e+03,2.400000e+03,2.023000e+03,1.000000e+00,1.645000e+03,4.983000e+03,2007.000000,1682.000000,1447.000000,816.000000,1648.000000


In [73]:
df_train = df[['fl_date','op_unique_carrier','tail_num','op_carrier_fl_num','origin_airport_id','dest_airport_id','crs_dep_time','crs_arr_time','crs_elapsed_time','distance']]

In [17]:
X01_test.to_csv('fl01_test')

In [12]:
X01_train, X01_test = train_test_split(df_01, test_size=0.2) 

In [145]:
df_train = X_train[['fl_date','op_unique_carrier','op_carrier_fl_num','arr_delay']]

In [178]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12591836 entries, 11774876 to 5066841
Data columns (total 3 columns):
 #   Column     Dtype  
---  ------     -----  
 0   fl_date    object 
 1   arr_delay  float64
 2   uni_fl     float64
dtypes: float64(2), object(1)
memory usage: 384.3+ MB


In [187]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12591836 entries, 11774876 to 5066841
Data columns (total 4 columns):
 #   Column       Dtype  
---  ------       -----  
 0   uni_fl       float64
 1   month        object 
 2   day_of_week  int64  
 3   is_holiday   int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 480.3+ MB


In [184]:
df_train.groupby('is_holiday')['arr_delay'].mean()

is_holiday
0    1.740948
1    1.362521
Name: arr_delay, dtype: float64

In [177]:
unifl_target_mean.to_csv('unifl_target_mean')

In [188]:
#choose set of features
df_train = X_train[['fl_date','op_unique_carrier','op_carrier_fl_num','arr_delay']]

# after test data, arr_delay should be 0. 
df_train['arr_delay'].fillna(0,inplace=True)

#remove outliner
df_train = df_train[(df_train['arr_delay'] > -80) & (df_train['arr_delay'] < 180)] 

#combine features
def combine_str_num(x,y):
    return x + str(y)
df_train['uni_fl'] = np.vectorize(combine_str_num)(df_train['op_unique_carrier'],df_train['op_carrier_fl_num'])
df_train.drop(columns=['op_unique_carrier','op_carrier_fl_num'],inplace=True)

# feature coding
unifl_target_mean = df_train.groupby(['uni_fl'])['arr_delay'].mean()
df_train['uni_fl'] = df_train['uni_fl'].map(unifl_target_mean)

D:\big data\anaconda3\envs\LH\lib\site-packages\pandas\core\generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [189]:
#transform fl_date
def getmonth(x):
    return x.split(sep='-')[1]
df_train['month'] = df_train['fl_date'].apply(getmonth)

def getdayofweek(x):
    year_s, mon_s, day_s = x.split('-')
    fl_d = datetime(int(year_s), int(mon_s), int(day_s))
    return fl_d.weekday()
df_train['day_of_week'] = df_train['fl_date'].apply(getdayofweek)

us_holidays = holidays.UnitedStates()
def isholiday(x):
    year_s, mon_s, day_s = x.split('-')
    if datetime(int(year_s), int(mon_s), int(day_s)) in us_holidays:
        return 1
    else:
        return 0
df_train['is_holiday'] = df_train['fl_date'].apply(isholiday)

y_train = df_train['arr_delay']

df_train = df_train.drop(columns=['fl_date','arr_delay'])

df_train = pd.get_dummies(df_train,columns=['month','day_of_week'])

In [208]:
scaler = StandardScaler()

In [209]:
X = scaler.fit_transform(df_train)

In [1]:
X.shape

NameError: name 'X' is not defined

In [190]:
y_train = df_train['arr_delay']

In [207]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12591836 entries, 11774876 to 5066841
Data columns (total 21 columns):
 #   Column         Dtype  
---  ------         -----  
 0   uni_fl         float64
 1   is_holiday     int64  
 2   month_01       uint8  
 3   month_02       uint8  
 4   month_03       uint8  
 5   month_04       uint8  
 6   month_05       uint8  
 7   month_06       uint8  
 8   month_07       uint8  
 9   month_08       uint8  
 10  month_09       uint8  
 11  month_10       uint8  
 12  month_11       uint8  
 13  month_12       uint8  
 14  day_of_week_0  uint8  
 15  day_of_week_1  uint8  
 16  day_of_week_2  uint8  
 17  day_of_week_3  uint8  
 18  day_of_week_4  uint8  
 19  day_of_week_5  uint8  
 20  day_of_week_6  uint8  
dtypes: float64(1), int64(1), uint8(19)
memory usage: 516.4 MB


In [197]:
df_train.head()

,uni_fl,is_holiday,month_01,month_02,month_03,month_04,month_05,month_06,month_07,month_08,...,month_10,month_11,month_12,day_of_week_0,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,day_of_week_6
11774876,4.592269,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
8648414,13.292593,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
11648639,5.556188,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
724728,-5.047619,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
3151471,2.464879,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [210]:
parametersGrid = {    "alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10],
                      "l1_ratio": [0,0.2,0,5,0.8]}

In [211]:
ent = ElasticNet()

In [ ]:
grid = GridSearchCV(ent, parametersGrid, scoring='r2', cv=5)
grid.fit(x_small, y_small)
#Y_pred = grid.predict(X_test)

D:\big data\anaconda3\envs\LH\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4336675303.313212, tolerance: 903289.7600536259
  model = cd_fast.enet_coordinate_descent(
